## Obtención de columnas con persistencia temporal y agrupación de Promedio, máximo y mínimo por hora.

#### Diseño del modelo

Una vez que se obtienen los datos limpios, estos pueden ser utilizados para una serie de pruebas, donde se decidirá la granularidad temporal y la representatividad geográfica con la que se va a efectuar el pronóstico. Este proceso es iterativo: se aplica un procesamiento a los datos, se entrenan los modelos y se analiza su desempeño. A continuación se muestran las principales variantes de los modelos generados así como sus descripción:


<img src="./fotos_markdown/diseno_modelo.png" style="height:300px">

* __Pronóstico por estación__: Se generó un modelo para todas las estaciónes de monitoreo. Debido a que la mayoría de las estaciones solo miden un subconjunto de todas las variables y no todas a la vez, los modelos generados sólo resultaron viables para las estaciones que miden todas las variables. Para la mayoría de las estaciones donde la cantidad de variables es baja,  el modelo subestima los valores de pronóstico.


* __Pronóstico para toda la Ciudad__: Para poder solventar que no todas las estaciones miden las mismas variables, se obtuvo el promedio, el mínimo y el máximo de los valores medidos en todas las estaciones por cada hora, al tomar estos valores, se pierde el atributo de la estación. El pronóstico mejoró, sin embargo el desempeño no era el esperado debido a los numerosos falsos positivos: la información-base de los modelos no era suficiente para describir el fenómeno.


* __Pronóstico por día__: Junto al proceso anterior, se procedió a volver a sacar el promedio, el mínimo y el máximo pero en esta ocasión por día y no por hora. Se obtuvo un modelo de pronóstico del valor máximo de cada día. El resultado no fue el esperado, el resultado mejoró considerablemente a comparación de los dos procesos anteriores sin embargo, el modelo seguía pronosticando demasiados falsos negativos: la contingencia no se pronosticaba correctamente debido a que al promediar por día, se pierde la información horaria y disminuye considerablemente el número de registros.


* __Pronóstico con persistencia temporal__: Usando los datos del pronóstico por hora para toda la ciudad, se utilizó la persistencia observada en el fenómeno entre horas pasadas y presentes, para generar nuevas columnas con los atributos de horas pasadas más correlacionadas con el presente. Se volvieron a entrenar los modelos con estas variables nuevas, obteniendo resultados satisfactorios.


__Pasos y descripción general del notebook__


El modelo elegido es el __horario con persistencia temporal__, habiendo obtenido la más baja tasa de falsos negativos y falsos positivos. Más adelante se expone su análisis con métricas de desempeño. A continuación se muestra el proceso para obtener los datos de entrenamiento del modelo:

<img src="./fotos_markdown/diagrama_2.png" style="height:350px">



1. __Contaminantes de horas anteriores más correlacionadas:__ se obtiene la correlación, de las variables meteorológicas y de contaminación, entre sus valores de una hora atrás con los de 72 horas atrás consigo mismas, de 1 hasta 72 horas atrás. El resultado es una una tabla de correlación temporal desplazada temporalmente la cual nos servirá para poder filtrar las variables más correlacionadas, es decir, las que más influyen en los niveles de contaminación.


2. __Aplicar Desplazamiento temporal:__ a partir de tablas de correlación temporal desplazada, se hace una columna para cada variable correlacionada: cuando existe una correlación mayor a 0.44 de cualquier atributo con los contaminantes mencionados, se hace una nueva columna con estos valores.



3. __Max, Mean, Min:__ se obtiene el máximo, el promedio y el mínimo de cada variable por hora, el resultado es una tabla con una sola fila por cada hora, en esta tabla se pierde el atributo de estación y así la información geográfica de donde se tomó la medición.


4. __Generar las columna a pronosticar:__ Se genera la columna con el valor del conntaminante a pronosticar a 24 horas y se obtiene la correlación de los valores promedio máximo y mínimo con éste. Por último se Filtran los valores más importantes.


5. __Generar las columnas de pronóstico de RH, WSP TMP:__ Se generan columnas con el pronóstico meteorológico  de estos atributos.


6. __Generar las columnas del contaminante a pronosticar de 1 hasta 23 horas:__ Se generan las columnas con el valor del contaminante a prononsticar desde 1 hasta 24 horas.



- __Datos recibidos:__ Limpieza de datos inicial
- __Responsable:__ Daniel Bustillos
- __Contacto:__  juandaniel.bucam@gmail.com

Este notebook presenta el proceso donde se ha obtenido los mejores desempeños de los modelos de los contaminantes. Después de un proceso iterativo y de un análisis del desempeño de los modelos, se obtuvieron las siguientes conclusiones:

- Utilizar la propiedad de persistencia temporal, mejora significativamente los modelos.


- Obtener el promedio, el máximo y el mínimo por hora en lugar de trabajar con la medición de cada estación, genera una consolidación de las mediciones y disminuye el efecto negativo de los atributos faltantes en las mediciones a raíz de las capacidades de las estación de monitoreo.


- Añadir el pronóstico meteorológico a 24 horas, añade fiabilidad a los modelos disminuyendo el error asociado del pronóstico. Este pronóstico se obtendrá posteriormente de una fuente externa.


In [1]:
Contaminantes = ["O3","PM10mean","PM25mean"]

In [2]:
contaminante = Contaminantes[0]
contaminante

'O3'

In [3]:
import pandas as pd
pd.options.mode.chained_assignment = None 

In [71]:
data_hour_merge_24 = pd.read_csv('./verificacion/datos_entrenamiento.csv')#verificationset

In [75]:
data_hour_merge_24.head()

,CO,NO,NO2,NOX,O3,PM10,PM2.5,RH,SO2,TMP,WSP,dia,fecha,hora,id_station,mes,PM10mean,PM25mean
0,3.5,NaN,NaN,NaN,6.0,NaN,NaN,NaN,32.0,NaN,NaN,1.0,2000-01-01 01:00:00,1,AZC,1.0,NaN,NaN
1,2.3,NaN,27.0,97.0,4.0,NaN,NaN,NaN,15.0,NaN,NaN,1.0,2000-01-01 01:00:00,1,BJU,1.0,NaN,NaN
2,2.6,NaN,56.0,110.0,8.0,69.0,NaN,36.0,NaN,9.7,0.1,1.0,2000-01-01 01:00:00,1,CES,1.0,NaN,NaN
3,NaN,NaN,NaN,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2000-01-01 01:00:00,1,CUA,1.0,NaN,NaN
4,2.4,NaN,35.0,119.0,4.0,NaN,NaN,NaN,9.0,10.7,0.2,1.0,2000-01-01 01:00:00,1,HAN,1.0,NaN,NaN


In [77]:
data_hour_merge_24 = data_hour_merge_24.dropna(subset=[contaminante]).reset_index(drop=True)

## 1. 
### Usando el archivo de correlación con retraso temporal, se crean nuevas columnas con el atributo y las horas atrás.


La corraloción con retraso temporal se refiere a la influencia que tienen las condiciones atmosféricas de un hora dada sobre las condiciones futuras, por ejemplo, si el nivel de contaminación de actual es alto ¿este valor influirá en la contaminación del día siguiente o la contaminación de cada día no se ve influida por valores pasados? La respuesta es la primera opción, valores pasados de las condiciones atmosféricas influyen fuertemente en la contaminación _actual_.

Esta propiedad es clave para la generación de un modelo estadístico, la existencia de patrones y de factores que aparentemente influyen en la contaminación nos permiten tener un conjunto de datos que determinen la contaminación.


La generación de la correlación se encuentra en: __/home/paw/DanielBustillos/contaminación/correlaciones_pau/correlaciones_función_paulina.ipynb__

Ya que tenemos los modelos entrenados, podemos definir el nombre de las columnas para posteriormente realizar el desplazamiento temporal:

Columnas con las que funcionan los modelos:

In [78]:
O3_columns_final = ['TMP_2.0_max',  'RH_22.0_max',  'hora_11.0',  'O3_47.0_max',  'RH_22.0_min',  'TMP_22.0_mean',  'O3_1.0_max',  'RH_48.0_mean',  'RH_max',  'RH_2.0_max',  'O3_2.0_max',  'O3_1.0_min',  'RH_23.0_min',  'RH_25.0_mean',  'TMP_mean_frcst_24',  'RH_0.0_mean',  'TMP_24.0_mean',  'hora_43.0',  'hora_20.0',  'O3_1.0_mean',  'TMP_mean',  'O3_44.0_mean',  'TMP_0.0_max',  'RH_0.0_max',  'O3_2.0_min',  'O3_0.0_max',  'O3_45.0_max',  'TMP_23.0_mean',  'RH_0.0_min',  'O3_23.0_max',  'RH_mean_frcst_22',  'TMP_24.0_max',  'RH_47.0_mean',  'TMP_mean_frcst_23',  'TMP_0.0_min',  'O3_24.0_mean',  'TMP_max',  'TMP_1.0_max',  'WSP_44.0_mean',  'O3_0.0_mean',  'O3_23.0_min',  'O3_46.0_max',  'RH_mean_frcst_21',  'RH_mean_frcst_1',  'RH_mean_frcst_2',  'TMP_21.0_max',  'TMP_mean_frcst_2',  'RH_2.0_min',  'TMP_46.0_mean',  'RH_24.0_min',  'TMP_45.0_mean',  'TMP_mean_frcst_3',  'TMP_mean_frcst_1',  'O3_min',  'O3_mean',  'O3_47.0_mean',  'TMP_2.0_mean',  'TMP_23.0_max',  'O3_2.0_mean',  'O3_21.0_max',  'RH_23.0_mean',  'RH_24.0_mean',  'O3_25.0_mean',  'O3_46.0_mean',  'RH_1.0_min',  'RH_45.0_mean',  'TMP_47.0_mean',  'RH_24.0_max',  'TMP_22.0_max',  'hora_19.0',  'O3_0.0_min',  'RH_mean_frcst_24',  'RH_mean',  'TMP_21.0_mean',  'RH_22.0_mean',  'RH_mean_frcst_3',  'RH_44.0_mean',  'TMP_min',  'TMP_0.0_mean',  'O3_24.0_max',  'hora_33.0',  'O3_21.0_mean',  'O3_22.0_min',  'hora_10.0',  'O3_22.0_mean',  'O3_24.0_min',  'RH_2.0_mean',  'RH_mean_frcst_23',  'O3_23.0_mean',  'O3_48.0_mean',  'RH_21.0_mean',  'RH_1.0_max',  'TMP_1.0_min',  'RH_1.0_mean',  'TMP_1.0_mean',  'RH_23.0_max',  'RH_46.0_mean',  'TMP_mean_frcst_22',  'hora_34.0',  'hora_9.0',  'O3_max',  'O3_45.0_mean',  'O3_22.0_max',  'RH_21.0_min',  'RH_min',  'WSP_mean']
PM10mean_columns_final = ['PM10mean_0.0_mean',  'PM10mean_20.0_mean',  'PM10_8.0_mean',  'PM10mean_3.0_max',  'PM10mean_25.0_max',  'O3_max',  'PM10mean_18.0_mean',  'PM25mean_2.0_max',  'PM10mean_9.0_max',  'PM10_1.0_mean',  'PM10mean_18.0_max',  'PM10mean_10.0_max',  'PM25mean_5.0_max',  'PM10mean_33.0_max',  'PM10mean_15.0_max',  'PM10mean_6.0_max',  'PM10mean_24.0_mean',  'PM10mean_max',  'PM10_1.0_max',  'PM10mean_1.0_max',  'PM10_10.0_mean',  'PM10mean_7.0_mean',  'PM10mean_24.0_max',  'PM10mean_21.0_mean',  'PM10mean_37.0_max',  'PM10mean_23.0_mean',  'PM25mean_0.0_mean',  'PM10_4.0_max',  'PM10mean_17.0_max',  'PM25mean_6.0_max',  'PM10mean_32.0_max',  'PM10_5.0_max',  'PM25mean_1.0_max',  'PM10mean_6.0_mean',  'PM10mean_11.0_mean',  'PM10mean_26.0_max',  'PM10mean_22.0_mean',  'PM10mean_16.0_mean',  'PM10mean_8.0_mean',  'PM25mean_4.0_max',  'PM10mean_30.0_max',  'PM25mean_0.0_max',  'PM10mean_23.0_max',  'NO2_max',  'WSP_mean',  'PM10mean_9.0_mean',  'PM10mean_17.0_mean',  'PM25mean_3.0_mean',  'PM25mean_7.0_max',  'PM10mean_16.0_max',  'PM10mean_14.0_mean',  'PM10mean_2.0_max',  'PM10mean_2.0_mean',  'PM10mean_mean',  'PM10_0.0_max',  'PM10mean_12.0_max',  'PM10mean_13.0_mean',  'PM10mean_8.0_max',  'PM10mean_25.0_mean',  'PM10mean_27.0_max',  'PM10mean_31.0_max',  'PM10mean_0.0_max',  'PM25mean_3.0_max',  'PM10mean_14.0_max',  'PM25mean_1.0_mean',  'PM10_2.0_max',  'PM10_0.0_mean',  'PM10mean_15.0_mean',  'PM10mean_19.0_mean',  'PM10mean_13.0_max',  'PM10mean_4.0_max',  'PM10_3.0_max',  'PM10mean_1.0_mean',  'PM10_7.0_max',  'PM10mean_20.0_max',  'PM25mean_8.0_max',  'PM10_5.0_mean',  'PM10mean_21.0_max',  'PM10mean_29.0_max',  'PM10mean_12.0_mean',  'PM10_4.0_mean',  'PM10_mean',  'RH_mean',  'PM10mean_5.0_max',  'PM2.5_max',  'PM10mean_4.0_mean',  'PM10_6.0_max',  'PM10mean_36.0_max',  'PM10mean_7.0_max',  'PM10_7.0_mean',  'PM25mean_mean',  'PM10_9.0_mean',  'PM25mean_9.0_max',  'PM10_3.0_mean',  'PM2.5_0.0_max',  'PM10mean_3.0_mean',  'PM10_6.0_mean',  'PM10mean_5.0_mean',  'PM10mean_22.0_max',  'PM10_max',  'PM25mean_10.0_max',  'PM10mean_35.0_max',  'PM25mean_max',  'PM10mean_19.0_max',  'PM10mean_10.0_mean',  'PM10mean_11.0_max',  'PM25mean_2.0_mean',  'TMP_mean',  'PM10mean_34.0_max',  'PM10_2.0_mean',  'PM10mean_28.0_max']
PM25mean_columns_final = ['PM25mean_14.0_max',  'PM10mean_0.0_mean',  'PM10mean_20.0_mean',  'PM10_8.0_mean',  'PM10mean_3.0_max',  'PM2.5_6.0_max',  'O3_max',  'PM10mean_18.0_mean',  'PM25mean_2.0_max',  'PM2.5_9.0_mean',  'PM10_1.0_mean',  'PM2.5_2.0_max',  'PM25mean_0.0_min',  'PM25mean_13.0_max',  'PM10mean_10.0_max',  'PM25mean_5.0_max',  'PM10mean_15.0_max',  'PM10mean_6.0_max',  'PM10mean_0.0_min',  'NO2_3.0_max',  'PM10mean_max',  'PM10_1.0_max',  'PM10mean_1.0_max',  'PM10_10.0_mean',  'PM2.5_4.0_mean',  'PM10mean_7.0_mean',  'PM10mean_21.0_mean',  'PM2.5_mean',  'PM10mean_23.0_mean',  'PM25mean_0.0_mean',  'PM10_4.0_max',  'PM10mean_17.0_max',  'PM10_12.0_mean',  'PM25mean_6.0_max',  'PM25mean_6.0_mean',  'PM25mean_1.0_max',  'PM10mean_6.0_mean',  'PM10mean_11.0_mean',  'PM2.5_3.0_max',  'PM2.5_2.0_mean',  'PM10mean_22.0_mean',  'PM10mean_16.0_mean',  'PM10mean_8.0_mean',  'PM25mean_11.0_mean',  'PM25mean_4.0_max',  'PM25mean_0.0_max',  'NO2_max',  'WSP_mean',  'PM10mean_9.0_mean',  'PM10mean_17.0_mean',  'PM2.5_8.0_mean',  'PM25mean_3.0_mean',  'PM25mean_7.0_max',  'PM25mean_9.0_mean',  'PM10mean_16.0_max',  'PM10mean_14.0_mean',  'NO2_mean',  'PM10mean_2.0_max',  'PM10mean_2.0_mean',  'PM10mean_mean',  'PM10_0.0_max',  'PM10mean_12.0_max',  'PM25mean_min',  'PM10mean_13.0_mean',  'PM10mean_8.0_max',  'PM10mean_0.0_max',  'PM25mean_3.0_max',  'PM10mean_14.0_max',  'PM25mean_1.0_mean',  'PM10_2.0_max',  'PM10_0.0_mean',  'PM10mean_15.0_mean',  'PM10mean_19.0_mean',  'PM10_11.0_mean',  'PM10mean_13.0_max',  'PM10mean_4.0_max',  'PM25mean_10.0_mean',  'PM25mean_14.0_mean',  'PM10_3.0_max',  'PM10mean_1.0_mean',  'PM2.5_1.0_mean',  'PM25mean_8.0_max',  'PM25mean_4.0_mean',  'PM10_5.0_mean',  'PM25mean_5.0_mean',  'PM25mean_11.0_max',  'PM10mean_12.0_mean',  'PM10_4.0_mean',  'PM10_mean',  'RH_mean',  'PM10mean_1.0_min',  'PM2.5_0.0_mean',  'PM10mean_5.0_max',  'PM2.5_max',  'PM10mean_4.0_mean',  'PM2.5_4.0_max',  'PM2.5_5.0_max',  'PM2.5_min',  'PM10mean_7.0_max',  'PM10_7.0_mean',  'PM25mean_mean',  'PM10_9.0_mean',  'PM25mean_9.0_max',  'PM10mean_min',  'PM10_3.0_mean',  'PM2.5_1.0_max',  'PM2.5_5.0_mean',  'PM2.5_6.0_mean',  'PM2.5_0.0_max',  'PM10mean_3.0_mean',  'PM10_6.0_mean',  'PM10mean_5.0_mean',  'PM25mean_8.0_mean',  'PM10_max',  'PM25mean_10.0_max',  'PM25mean_max',  'PM25mean_12.0_max',  'PM25mean_1.0_min',  'PM2.5_3.0_mean',  'PM25mean_15.0_mean',  'PM10mean_10.0_mean',  'PM25mean_12.0_mean',  'PM10mean_11.0_max',  'PM25mean_2.0_mean',  'TMP_mean',  'PM2.5_7.0_mean',  'PM25mean_7.0_mean',  'PM25mean_13.0_mean',  'PM10_2.0_mean',  'PM10mean_9.0_max',  'PM2.5_0.0_min']

Columnas con losque se filtra inicialmente:

In [79]:
PM25mean_columns_filter = ['PM25mean_0.0',  'PM10_0.0',  'PM2.5_0.0',  'PM10mean_0.0',  'PM25mean_0.0',  'PM10mean_1.0',  'PM10mean_2.0',  'PM10mean_3.0',  'PM10mean_4.0',  'PM10mean_5.0',  'PM10mean_6.0',  'PM10mean_7.0',  'PM10mean_8.0',  'PM10mean_9.0',  'PM10mean_10.0',  'PM10mean_11.0',  'PM10mean_12.0',  'PM10mean_13.0',  'PM10mean_14.0',  'PM10mean_15.0',  'PM10mean_16.0',  'PM25mean_1.0',  'PM25mean_2.0',  'PM25mean_3.0',  'PM25mean_4.0',  'PM25mean_5.0',  'PM25mean_6.0',  'PM25mean_7.0',  'PM25mean_8.0',  'PM25mean_9.0',  'PM25mean_10.0',  'PM25mean_11.0',  'PM25mean_12.0',  'PM25mean_13.0',  'PM25mean_14.0',  'PM25mean_15.0',  'PM25mean_16.0',  'PM25mean_17.0',  'PM25mean_18.0',  'PM10mean_1.0',  'PM10mean_2.0',  'PM10mean_3.0',  'PM10mean_4.0',  'PM10mean_5.0',  'PM10mean_6.0',  'PM10mean_7.0',  'PM10mean_8.0',  'PM10mean_9.0',  'PM10mean_10.0',  'PM10mean_11.0',  'PM10mean_12.0',  'PM10mean_13.0',  'PM10mean_14.0',  'PM10mean_15.0',  'PM10mean_16.0',  'PM10mean_17.0',  'PM10mean_18.0',  'PM10mean_19.0',  'PM10mean_20.0',  'PM10mean_21.0',  'PM10mean_22.0',  'PM10mean_23.0',  'PM10mean_24.0',  'PM10mean_25.0',  'PM10mean_26.0',  'PM10mean_27.0',  'PM10mean_28.0',  'PM10mean_29.0',  'PM10mean_30.0',  'PM10mean_31.0',  'PM10mean_32.0',  'PM10mean_33.0',  'PM10mean_34.0',  'PM10mean_35.0',  'PM10mean_36.0',  'PM10mean_37.0',  'PM10mean_38.0',  'PM25mean_1.0',  'PM25mean_2.0',  'PM25mean_3.0',  'PM25mean_4.0',  'PM25mean_5.0',  'PM25mean_6.0',  'PM25mean_7.0',  'PM25mean_8.0',  'PM25mean_9.0',  'PM25mean_10.0',  'PM25mean_11.0',  'PM25mean_12.0',  'PM25mean_13.0',  'PM25mean_14.0',  'PM25mean_15.0',  'PM25mean_16.0',  'PM25mean_17.0',  'PM25mean_18.0',  'PM25mean_19.0',  'PM25mean_20.0',  'PM25mean_21.0',  'PM25mean_22.0',  'PM25mean_23.0',  'PM25mean_24.0',  'PM25mean_25.0',  'PM25mean_26.0',  'PM25mean_27.0',  'PM25mean_28.0',  'PM25mean_29.0',  'PM25mean_30.0',  'PM25mean_31.0',  'PM25mean_32.0',  'PM25mean_33.0',  'PM25mean_34.0',  'PM25mean_35.0',  'PM25mean_36.0',  'PM25mean_37.0',  'NO2_3.0',  'NO2_4.0',  'NO2_5.0',  'NO2_6.0',  'NO2_7.0',  'NO2_8.0',  'NO2_9.0',  'NO2_10.0',  'NO2_11.0',  'NO2_12.0',  'NO2_13.0',  'NO2_14.0',  'NO2_15.0',  'NO2_16.0',  'NO2_17.0',  'NO2_18.0',  'NO2_19.0',  'NO2_20.0',  'NO2_21.0',  'NO2_22.0',  'NO2_23.0',  'PM10_1.0',  'PM10_2.0',  'PM10_3.0',  'PM10_4.0',  'PM10_5.0',  'PM10_6.0',  'PM10_7.0',  'PM10_8.0',  'PM10_9.0',  'PM10_10.0',  'PM10_11.0',  'PM10_12.0',  'PM10_13.0',  'PM10_14.0',  'PM10_15.0',  'PM10_16.0',  'PM10_17.0',  'PM10_18.0',  'PM10_19.0',  'PM10_20.0',  'PM10_21.0',  'PM10_22.0',  'PM10_23.0',  'PM10_24.0',  'PM10_25.0',  'PM10_26.0',  'PM10_27.0',  'PM10_28.0',  'PM10_29.0',  'PM10_30.0',  'PM2.5_1.0',  'PM2.5_2.0',  'PM2.5_3.0',  'PM2.5_4.0',  'PM2.5_5.0',  'PM2.5_6.0',  'PM2.5_7.0',  'PM2.5_8.0',  'PM2.5_9.0',  'PM2.5_10.0',  'PM2.5_11.0',  'PM2.5_12.0',  'PM2.5_13.0',  'PM2.5_14.0',  'PM2.5_15.0',  'PM2.5_16.0',  'PM2.5_17.0',  'PM2.5_18.0',  'PM2.5_19.0',  'PM2.5_20.0',  'PM2.5_21.0',  'PM2.5_22.0',  'PM2.5_23.0',  'PM2.5_24.0',  'PM2.5_25.0',  'PM2.5_26.0',  'PM2.5_27.0',  'PM2.5_28.0',  'PM10mean_1.0',  'PM10mean_2.0',  'PM10mean_3.0',  'PM10mean_4.0',  'PM10mean_5.0',  'PM10mean_6.0',  'PM10mean_7.0',  'PM10mean_8.0',  'PM10mean_9.0',  'PM10mean_10.0',  'PM10mean_11.0',  'PM10mean_12.0',  'PM10mean_13.0',  'PM10mean_14.0',  'PM10mean_15.0',  'PM10mean_16.0',  'PM10mean_17.0',  'PM10mean_18.0',  'PM10mean_19.0',  'PM10mean_20.0',  'PM10mean_21.0',  'PM10mean_22.0',  'PM10mean_23.0',  'PM10mean_24.0',  'PM10mean_25.0',  'PM10mean_26.0',  'PM10mean_27.0',  'PM10mean_28.0',  'PM10mean_29.0',  'PM10mean_30.0',  'PM10mean_31.0',  'PM10mean_32.0',  'PM10mean_33.0',  'PM10mean_34.0',  'PM10mean_35.0',  'PM10mean_36.0',  'PM10mean_37.0',  'PM10mean_38.0',  'PM10mean_39.0',  'PM10mean_40.0',  'PM10mean_41.0',  'PM10mean_42.0',  'PM25mean_1.0',  'PM25mean_2.0',  'PM25mean_3.0',  'PM25mean_4.0',  'PM25mean_5.0',  'PM25mean_6.0',  'PM25mean_7.0',  'PM25mean_8.0',  'PM25mean_9.0',  'PM25mean_10.0',  'PM25mean_11.0',  'PM25mean_12.0',  'PM25mean_13.0',  'PM25mean_14.0',  'PM25mean_15.0',  'PM25mean_16.0',  'PM25mean_17.0',  'PM25mean_18.0',  'PM25mean_19.0',  'PM25mean_20.0',  'PM25mean_21.0',  'PM25mean_22.0',  'PM25mean_23.0',  'PM25mean_24.0',  'PM25mean_25.0',  'PM25mean_26.0',  'PM25mean_27.0',  'PM25mean_28.0',  'PM25mean_29.0',  'PM25mean_30.0',  'PM25mean_31.0',  'PM25mean_32.0',  'PM25mean_33.0',  'PM25mean_34.0']
PM10mean_columns_filter = ['PM10_0.0',  'PM2.5_0.0',  'PM10mean_0.0',  'PM25mean_0.0',  'PM10mean_1.0',  'PM10mean_2.0',  'PM10mean_3.0',  'PM10mean_4.0',  'PM10mean_5.0',  'PM10mean_6.0',  'PM10mean_7.0',  'PM10mean_8.0',  'PM10mean_9.0',  'PM10mean_10.0',  'PM10mean_11.0',  'PM10mean_12.0',  'PM10mean_13.0',  'PM10mean_14.0',  'PM10mean_15.0',  'PM25mean_1.0',  'PM25mean_2.0',  'PM10mean_1.0',  'PM10mean_2.0',  'PM10mean_3.0',  'PM10mean_4.0',  'PM10mean_5.0',  'PM10mean_6.0',  'PM10mean_7.0',  'PM10mean_8.0',  'PM10mean_9.0',  'PM10mean_10.0',  'PM10mean_11.0',  'PM10mean_12.0',  'PM10mean_13.0',  'PM10mean_14.0',  'PM10mean_15.0',  'PM10mean_16.0',  'PM10mean_17.0',  'PM10mean_18.0',  'PM10mean_19.0',  'PM10mean_20.0',  'PM10mean_21.0',  'PM10mean_22.0',  'PM10mean_23.0',  'PM10mean_24.0',  'PM10mean_25.0',  'PM10mean_26.0',  'PM10mean_27.0',  'PM10mean_28.0',  'PM10mean_29.0',  'PM10mean_30.0',  'PM10mean_31.0',  'PM10mean_32.0',  'PM10mean_33.0',  'PM10mean_34.0',  'PM10mean_35.0',  'PM10mean_36.0',  'PM10mean_37.0',  'PM10mean_38.0',  'PM10mean_39.0',  'PM10mean_40.0',  'PM10mean_41.0',  'PM10mean_42.0',  'PM10mean_43.0',  'PM10mean_44.0',  'PM10mean_45.0',  'PM10mean_46.0',  'PM10mean_47.0',  'PM10mean_48.0',  'PM25mean_1.0',  'PM25mean_2.0',  'PM25mean_3.0',  'PM25mean_4.0',  'PM25mean_5.0',  'PM25mean_6.0',  'PM25mean_7.0',  'PM25mean_8.0',  'PM25mean_9.0',  'PM25mean_10.0',  'PM25mean_11.0',  'PM25mean_12.0',  'PM25mean_13.0',  'PM25mean_14.0',  'PM25mean_15.0',  'PM25mean_16.0',  'PM25mean_17.0',  'PM25mean_18.0',  'PM25mean_19.0',  'PM25mean_20.0',  'PM25mean_21.0',  'PM25mean_22.0',  'PM25mean_23.0',  'PM25mean_24.0',  'PM25mean_25.0',  'PM25mean_26.0',  'PM25mean_27.0',  'PM25mean_28.0',  'PM25mean_29.0',  'PM25mean_30.0',  'PM25mean_31.0',  'PM25mean_32.0',  'PM25mean_33.0',  'PM25mean_34.0',  'PM25mean_35.0',  'PM25mean_36.0',  'PM25mean_37.0',  'NO2_5.0',  'NO2_6.0',  'NO2_7.0',  'NO2_8.0',  'NO2_9.0',  'NO2_10.0',  'NO2_11.0',  'NO2_12.0',  'NO2_13.0',  'NO2_14.0',  'NO2_15.0',  'NO2_16.0',  'NO2_17.0',  'NO2_18.0',  'NO2_19.0',  'NO2_20.0',  'NO2_21.0',  'NO2_22.0',  'NO2_23.0',  'PM10_1.0',  'PM10_2.0',  'PM10_3.0',  'PM10_4.0',  'PM10_5.0',  'PM10_6.0',  'PM10_7.0',  'PM10_8.0',  'PM10_9.0',  'PM10_10.0',  'PM10_11.0',  'PM10_12.0',  'PM10_13.0',  'PM10_14.0',  'PM10_15.0',  'PM10_16.0',  'PM10_17.0',  'PM10_18.0',  'PM10_19.0',  'PM10_20.0',  'PM10_21.0',  'PM10_22.0',  'PM10_23.0',  'PM10_24.0',  'PM10_25.0',  'PM10_26.0',  'PM10_27.0',  'PM10_28.0',  'PM10_29.0',  'PM10_30.0',  'PM10_31.0',  'PM10_32.0',  'PM10_33.0',  'PM10_34.0',  'PM10_35.0',  'PM10_36.0',  'PM2.5_1.0',  'PM2.5_2.0',  'PM2.5_3.0',  'PM2.5_4.0',  'PM2.5_5.0',  'PM2.5_6.0',  'PM2.5_7.0',  'PM2.5_8.0',  'PM2.5_9.0',  'PM2.5_10.0',  'PM2.5_11.0',  'PM2.5_12.0',  'PM2.5_13.0',  'PM2.5_14.0',  'PM2.5_15.0',  'PM2.5_16.0',  'PM2.5_17.0',  'PM2.5_18.0',  'PM2.5_19.0',  'PM2.5_20.0',  'PM2.5_21.0',  'PM2.5_22.0',  'PM2.5_23.0',  'PM10mean_1.0',  'PM10mean_2.0',  'PM10mean_3.0',  'PM10mean_4.0',  'PM10mean_5.0',  'PM10mean_6.0',  'PM10mean_7.0',  'PM10mean_8.0',  'PM10mean_9.0',  'PM10mean_10.0',  'PM10mean_11.0',  'PM10mean_12.0',  'PM10mean_13.0',  'PM10mean_14.0',  'PM10mean_15.0',  'PM10mean_16.0',  'PM10mean_17.0',  'PM10mean_18.0',  'PM10mean_19.0',  'PM10mean_20.0',  'PM10mean_21.0',  'PM10mean_22.0',  'PM10mean_23.0',  'PM10mean_24.0',  'PM10mean_25.0',  'PM10mean_26.0',  'PM10mean_27.0',  'PM10mean_28.0',  'PM10mean_29.0',  'PM10mean_30.0',  'PM10mean_31.0',  'PM10mean_32.0',  'PM10mean_33.0',  'PM10mean_34.0',  'PM10mean_35.0',  'PM10mean_36.0',  'PM10mean_37.0',  'PM10mean_38.0',  'PM10mean_39.0',  'PM10mean_40.0',  'PM10mean_41.0',  'PM10mean_42.0',  'PM10mean_43.0',  'PM10mean_44.0',  'PM10mean_45.0',  'PM10mean_46.0',  'PM10mean_47.0',  'PM10mean_48.0',  'PM25mean_1.0',  'PM25mean_2.0',  'PM25mean_3.0',  'PM25mean_4.0',  'PM25mean_5.0',  'PM25mean_6.0',  'PM25mean_7.0',  'PM25mean_8.0',  'PM25mean_9.0',  'PM25mean_10.0',  'PM25mean_11.0',  'PM25mean_12.0',  'PM25mean_13.0',  'PM25mean_14.0',  'PM25mean_15.0',  'PM25mean_16.0',  'PM25mean_17.0',  'PM25mean_18.0',  'PM25mean_19.0',  'PM25mean_20.0',  'PM25mean_21.0',  'PM25mean_22.0',  'PM25mean_23.0',  'PM25mean_24.0',  'PM25mean_25.0',  'PM25mean_26.0',  'PM25mean_27.0',  'PM25mean_28.0',  'PM25mean_29.0']
O3_columns_filter = ['O3_0.0',  'RH_0.0',  'TMP_0.0',  'hora_9.0',  'hora_10.0',  'hora_11.0',  'hora_12.0',  'hora_13.0',  'hora_18.0',  'hora_19.0',  'hora_20.0',  'hora_21.0',  'hora_22.0',  'hora_32.0',  'hora_33.0',  'hora_34.0',  'hora_35.0',  'hora_36.0',  'hora_42.0',  'hora_43.0',  'hora_44.0',  'hora_45.0',  'O3_1.0',  'O3_2.0',  'O3_3.0',  'O3_4.0',  'O3_21.0',  'O3_22.0',  'O3_23.0',  'O3_24.0',  'O3_25.0',  'O3_26.0',  'O3_27.0',  'O3_45.0',  'O3_46.0',  'O3_47.0',  'O3_48.0',  'O3_1.0',  'O3_2.0',  'O3_3.0',  'O3_4.0',  'O3_20.0',  'O3_21.0',  'O3_22.0',  'O3_23.0',  'O3_24.0',  'O3_25.0',  'O3_26.0',  'O3_27.0',  'O3_44.0',  'O3_45.0',  'O3_46.0',  'O3_47.0',  'O3_48.0',  'CO_5.0',  'CO_6.0',  'CO_7.0',  'CO_8.0',  'CO_29.0',  'CO_30.0',  'CO_31.0',  'NO_6.0',  'NO_7.0',  'NO_8.0',  'NO_9.0',  'NO_30.0',  'NO_31.0',  'NO_32.0',  'NOX_6.0',  'NOX_7.0',  'NOX_8.0',  'NOX_9.0',  'NOX_30.0',  'NOX_31.0',  'NOX_32.0',  'O3_1.0',  'O3_2.0',  'O3_3.0',  'O3_4.0',  'O3_21.0',  'O3_22.0',  'O3_23.0',  'O3_24.0',  'O3_25.0',  'O3_26.0',  'O3_27.0',  'O3_44.0',  'O3_45.0',  'O3_46.0',  'O3_47.0',  'O3_48.0',  'PM10_1.0',  'PM10_2.0',  'PM10_3.0',  'PM10_4.0',  'PM10_5.0',  'PM10_6.0',  'PM10_26.0',  'PM10_27.0',  'PM10_28.0',  'PM10_29.0',  'PM2.5_1.0',  'PM2.5_2.0',  'PM2.5_3.0',  'PM2.5_4.0',  'PM2.5_5.0',  'PM2.5_6.0',  'PM2.5_26.0',  'PM2.5_27.0',  'PM2.5_28.0',  'PM2.5_29.0',  'RH_1.0',  'RH_2.0',  'RH_3.0',  'RH_20.0',  'RH_21.0',  'RH_22.0',  'RH_23.0',  'RH_24.0',  'RH_25.0',  'RH_26.0',  'RH_44.0',  'RH_45.0',  'RH_46.0',  'RH_47.0',  'RH_48.0',  'TMP_1.0',  'TMP_2.0',  'TMP_3.0',  'TMP_20.0',  'TMP_21.0',  'TMP_22.0',  'TMP_23.0',  'TMP_24.0',  'TMP_25.0',  'TMP_26.0',  'TMP_44.0',  'TMP_45.0',  'TMP_46.0',  'TMP_47.0',  'TMP_48.0',  'WSP_8.0',  'WSP_9.0',  'WSP_10.0',  'WSP_11.0',  'WSP_44.0',  'WSP_45.0']

In [80]:
if contaminante == "O3":
    columns = O3_columns_final
    columns_filter = O3_columns_filter
elif contaminante=="PM10mean":
    columns = PM10mean_columns_final
    columns_filter = PM10mean_columns_filter
elif contaminante=="PM25mean":
    columns = PM25mean_columns_final
    columns_filter = PM25mean_columns_filter

In [81]:
columns_filter

['O3_0.0',
 'RH_0.0',
 'TMP_0.0',
 'hora_9.0',
 'hora_10.0',
 'hora_11.0',
 'hora_12.0',
 'hora_13.0',
 'hora_18.0',
 'hora_19.0',
 'hora_20.0',
 'hora_21.0',
 'hora_22.0',
 'hora_32.0',
 'hora_33.0',
 'hora_34.0',
 'hora_35.0',
 'hora_36.0',
 'hora_42.0',
 'hora_43.0',
 'hora_44.0',
 'hora_45.0',
 'O3_1.0',
 'O3_2.0',
 'O3_3.0',
 'O3_4.0',
 'O3_21.0',
 'O3_22.0',
 'O3_23.0',
 'O3_24.0',
 'O3_25.0',
 'O3_26.0',
 'O3_27.0',
 'O3_45.0',
 'O3_46.0',
 'O3_47.0',
 'O3_48.0',
 'O3_1.0',
 'O3_2.0',
 'O3_3.0',
 'O3_4.0',
 'O3_20.0',
 'O3_21.0',
 'O3_22.0',
 'O3_23.0',
 'O3_24.0',
 'O3_25.0',
 'O3_26.0',
 'O3_27.0',
 'O3_44.0',
 'O3_45.0',
 'O3_46.0',
 'O3_47.0',
 'O3_48.0',
 'CO_5.0',
 'CO_6.0',
 'CO_7.0',
 'CO_8.0',
 'CO_29.0',
 'CO_30.0',
 'CO_31.0',
 'NO_6.0',
 'NO_7.0',
 'NO_8.0',
 'NO_9.0',
 'NO_30.0',
 'NO_31.0',
 'NO_32.0',
 'NOX_6.0',
 'NOX_7.0',
 'NOX_8.0',
 'NOX_9.0',
 'NOX_30.0',
 'NOX_31.0',
 'NOX_32.0',
 'O3_1.0',
 'O3_2.0',
 'O3_3.0',
 'O3_4.0',
 'O3_21.0',
 'O3_22.0',
 'O3_23.0'

## 1. 
### Usando el archivo de correlación con retraso temporal, se crean nuevas columnas con el atributo y las horas atrás.


La corraloción con retraso temporal se refiere a la influencia que tienen las condiciones atmosféricas de un hora dada sobre las condiciones futuras, por ejemplo, si el nivel de contaminación de actual es alto ¿este valor influirá en la contaminación del día siguiente o la contaminación de cada día no se ve influida por valores pasados? La respuesta es la primera opción, valores pasados de las condiciones atmosféricas influyen fuertemente en la contaminación _actual_.

Esta propiedad es clave para la generación de un modelo estadístico, la existencia de patrones y de factores que aparentemente influyen en la contaminación nos permiten tener un conjunto de datos que determinen la contaminación.


La generación de la correlación se encuentra en: __/home/paw/DanielBustillos/contaminación/correlaciones_pau/correlaciones_función_paulina.ipynb__

Ya que tenemos los modelos entrenados, podemos definir el nombre de las columnas para posteriormente realizar el desplazamiento temporal:

In [82]:
O3_wo_end = columns_filter

In [83]:
nombres_items = [item.split("_")[0] for item in O3_wo_end]
horas_items = [item.split("_")[1] for item in O3_wo_end]

Una vez que tenemos los contaminantes y el número de horas de retraso, definimos una función que nos genere una nueva columna con el contaminante recorrido temporalmente las horas indicadas en **O3_corr_filtro**:

In [84]:
def shit_corr(df):
    for i in range(len(O3_wo_end)):
        name_column = str(nombres_items[i] + "_" + str(horas_items[i]))
                                                # se define el nombre de la columna a crear con el formato:
                                            # contaminante _ horas de retraso
        df[name_column] = df[nombres_items[i]].shift(
            int(float(str(horas_items[i])))) # se toma el nombre del contaminante y se 
                                                            # desplaza temporalmente las horas indicadas
    return df.dropna()

Debido a que por cada hora se tienen tantos registros como el número de estaciones de monitoreo, existen fechas duplicadas en el df:

Al tener fechas duplicadas, puede ocurrrir errores al aplicar el desplazamiento, por este motivo vamos a generar **un dataframe por cada estación**, aplicaremos la función y por último uniremos los dataframes:

In [85]:
estaciones = data_hour_merge_24.id_station.unique().tolist() # recolectamos los nombres de las estaciones
data_est = {}
for elem in estaciones:
    data_est[elem] = data_hour_merge_24[data_hour_merge_24.id_station == elem] #hacemos un df por estación

Aplicamos la función para cada dataframe:

In [86]:
data_shift = {}
for elem in data_est:
    data_shift[elem] = shit_corr(data_est[elem])

Unimos los dataframes:

In [87]:
df_append = pd.DataFrame(columns = data_est["MER"].columns.tolist())
for key in data_est:
    df_append = df_append.append(data_est[key], ignore_index=True,sort=True)

# 2.
### Se obtiene el promedio, máximo y mínimo por hora de todas las estaciones.

Debido a que solo algunas estaciones miden todos los atributos con los que se está trabajando, se encontró que agrupar las condiciones atmosféricas de cada atributo con el promedio, máximo y mínimo, genera una mejora considerable al desempeño final de los modelos, debido a que al hacer esta agrupación se suavizan los valores y también se homogeniza los atributos de cada estación.

Se guardan las columnas al aplicar el groupby y se elimina "fecha":

In [91]:
cols = df_append.columns.tolist() 
cols.remove("fecha")
cols.remove("id_station")
df_append[cols] = df_append[cols].astype(float)

Aplicamos los groupbys:

In [92]:
data_hour_merge_24_max = df_append.groupby('fecha')[cols].max()
data_hour_merge_24_max.reset_index(inplace=True)

In [93]:
data_hour_merge_24_min = df_append.groupby('fecha')[cols].min()
data_hour_merge_24_min.reset_index(inplace=True)

In [94]:
data_hour_merge_24_mean = df_append.groupby('fecha')[cols].mean()
data_hour_merge_24_mean.reset_index(inplace=True)

#### Cambiamos los nombres de las estaciones:
Debido a que cada groupby genera los mismos nombres, esto puede generar problemas para identificar de qué columna se trata, se cambiarán los nombres a "contaminante_max", "contaminante_min", "contaminante_mean" según corresponda.

Se obtienenn los nonbres de las columnas en forma de lista:

In [95]:
max_columns = data_hour_merge_24_max.columns.tolist()
min_columns = data_hour_merge_24_min.columns.tolist()
mean_columns = data_hour_merge_24_mean.columns.tolist()

Se cambian los nombres en las listas:

In [96]:
for i in range(len(max_columns)):
    if not max_columns[i].startswith("fecha") \
    and not max_columns[i].startswith("hora") \
    and not max_columns[i].startswith("dia") \
    and not max_columns[i].startswith("mes"):
        max_columns[i] = max_columns[i]+"_max"
        
    if not min_columns[i].startswith("fecha") \
    and not min_columns[i].startswith("hora") \
    and not min_columns[i].startswith("dia") \
    and not min_columns[i].startswith("mes"):
        min_columns[i] = min_columns[i]+"_min" 
        
    if not mean_columns[i].startswith("fecha") \
    and not mean_columns[i].startswith("hora") \
    and not mean_columns[i].startswith("dia") \
    and not mean_columns[i].startswith("mes"):
        mean_columns[i] = mean_columns[i]+"_mean"

Se renombra las columnas de los dataframes

In [97]:
data_hour_merge_24_mean.columns = mean_columns
data_hour_merge_24_min.columns = min_columns
data_hour_merge_24_max.columns = max_columns

Unimos los df's 

In [98]:
data_hour_merge = pd.merge(data_hour_merge_24_mean, data_hour_merge_24_max, on=['fecha'])
data_hour_merge = pd.merge(data_hour_merge, data_hour_merge_24_min, on=['fecha'])

In [99]:
data_hour_merge_columns = data_hour_merge.columns.tolist()

Debido a que el proceso de obtener el máximo y mínimo es lento, es conveniente guardar el archivo de manera provisional:

# Arreglemos los nnombres:

In [102]:
for item in range(len(data_hour_merge_columns)):
    if  data_hour_merge_columns[item].endswith("_x")\
    or  data_hour_merge_columns[item].endswith("_y"):
        data_hour_merge_columns[item] = data_hour_merge_columns[item][:-2]

In [103]:
data_hour_merge.columns = data_hour_merge_columns#.tolist()

data_hour_merge = data_hour_merge.loc[:,~data_hour_merge.columns.duplicated()]

In [104]:
data_hour_merge = data_hour_merge.sort_values(['fecha',"hora"], ascending=[0,1]).reset_index(drop=True)

## 3. 
### Generar las columna a pronosticar y obtener la correlación de los valores promedio máximo y mínimo con éste. Filtrar los valores más importantes.


Ahora vamos a sacar los atributos más correlacionado con el contaminante a pronosticar a pronosticar. 

In [123]:
data_hour_merge.head()

,fecha,CO_mean,NO_mean,NO2_mean,NO2_10.0_mean,NO2_11.0_mean,NO2_12.0_mean,NO2_13.0_mean,NO2_14.0_mean,NO2_15.0_mean,...,PM25mean_4.0_min,PM25mean_5.0_min,PM25mean_6.0_min,PM25mean_7.0_min,PM25mean_8.0_min,PM25mean_9.0_min,RH_min,SO2_min,TMP_min,WSP_min
0,2019-09-06 17:00:00,0.523333,3.0,15.714286,32.714286,27.571429,21.428571,16.000000,13.714286,12.857143,...,16.916667,16.500000,16.125000,15.875000,15.375000,15.208333,24.0,1.0,20.0,1.4
1,2019-09-06 16:00:00,0.533333,1.6,13.714286,27.571429,21.428571,16.000000,13.714286,12.857143,13.571429,...,16.500000,16.125000,15.875000,15.375000,15.208333,15.125000,27.0,1.0,20.0,1.6
2,2019-09-06 15:00:00,0.601667,2.0,16.571429,21.428571,16.000000,13.714286,12.857143,13.571429,13.857143,...,16.125000,15.875000,15.375000,15.208333,15.125000,14.875000,32.0,1.0,20.0,1.9
3,2019-09-06 14:00:00,0.660000,2.6,19.857143,16.000000,13.714286,12.857143,13.571429,13.857143,14.571429,...,15.875000,15.375000,15.208333,15.125000,14.875000,15.166667,34.0,1.0,19.0,1.6
4,2019-09-06 13:00:00,0.656667,3.2,21.285714,13.714286,12.857143,13.571429,13.857143,14.571429,17.285714,...,15.375000,15.208333,15.125000,14.875000,15.166667,15.000000,37.0,2.0,19.0,1.8


## 4. 
### Generar las columnas de pronóstico de RH, WSP TMP.

Se observó que usando datos de pronóstico atmosférico, el modelo tiene un mejor desempeño, vamos a añadir "RH_mean","WSP_mean","TMP_mean" a los atributos con un pronóstico de 24 horas.

In [124]:
lista_frcst = ["RH_mean","WSP_mean","TMP_mean"]

In [125]:
for item_pronostico in lista_frcst:
    for i in range(1, 25):
        col_name = str(item_pronostico + "_frcst_" + str(i))
        data_hour_merge[col_name] = data_hour_merge[item_pronostico].shift(i)
data_hour_merge.dropna(inplace=True)

volvamos a sacar la correlación y filtramos los valores de correlación:

In [126]:
data_hour_merge.head(2)

,fecha,CO_mean,NO_mean,NO2_mean,NO2_10.0_mean,NO2_11.0_mean,NO2_12.0_mean,NO2_13.0_mean,NO2_14.0_mean,NO2_15.0_mean,...,TMP_mean_frcst_15,TMP_mean_frcst_16,TMP_mean_frcst_17,TMP_mean_frcst_18,TMP_mean_frcst_19,TMP_mean_frcst_20,TMP_mean_frcst_21,TMP_mean_frcst_22,TMP_mean_frcst_23,TMP_mean_frcst_24
24,2019-09-05 17:00:00,0.576667,2.8,17.857143,32.500000,30.166667,25.5,20.000000,19.428571,21.0,...,16.000000,16.142857,17.857143,19.142857,20.428571,21.142857,21.714286,22.428571,22.714286,22.714286
25,2019-09-05 16:00:00,0.541667,2.2,15.000000,30.166667,25.500000,20.0,19.428571,21.000000,22.5,...,15.428571,16.000000,16.142857,17.857143,19.142857,20.428571,21.142857,21.714286,22.428571,22.714286


### Filtramos

In [127]:
columns.append("fecha")

In [129]:
data_hour_merge = data_hour_merge[columns]

In [130]:
path_final = str("./datos/ingesta_modelos_"+ contaminante + ".csv")

In [131]:
data_hour_merge.to_csv("./datos/datos_modelos_"+ contaminante + ".csv",sep=',', encoding='utf-8',index=False)

# Función

In [12]:
import pandas as pd
pd.options.mode.chained_assignment = None     

In [13]:
contaminantes = ["O3","PM10mean","PM25mean"]

In [20]:
shif_contaminante(Contaminantes[2])

PM25mean_max


In [15]:
def shif_contaminante(contaminante):

    #data_hour_merge_24 = pd.read_csv('./datos/ingesta_contaminacion.csv')
    data_hour_merge_24 = pd.read_csv('./verificacion/datos_entrenamiento_verificacion.csv')#verificationset

    data_hour_merge_24 = data_hour_merge_24.dropna(subset=[contaminante]).reset_index(drop=True)#PM25
    ## 1. 
    ### Usando el archivo de correlación con retraso temporal, se crean nuevas columnas con el atributo y #
    #las horas atrás.


    #La corraloción con retraso temporal se refiere a la influencia que tienen las condiciones atmosféricas #
    #de un hora dada sobre las condiciones futuras, por ejemplo, si el nivel de contaminación de actual es
    #alto ¿este valor influirá en la contaminación del día siguiente o la contaminación de cada día no se 
    #ve influida por valores pasados? La respuesta es la primera opción, valores pasados de las condiciones 
    #atmosféricas influyen fuertemente en la contaminación _actual_.

    #Esta propiedad es clave para la generación de un modelo estadístico, la existencia de patrones y de 
    #factores que aparentemente influyen en la contaminación nos permiten tener un conjunto de datos que#
    #determinen la contaminación.

    #Ya que tenemos los modelos entrenados, podemos definir el nombre de las columnas para #
    #posteriormente realizar el desplazamiento temporal:

    #Columnas con las que funcionan los modelos:

    O3_columns_final = ['TMP_2.0_max',  'RH_22.0_max',  'hora_11.0',  'O3_47.0_max',  'RH_22.0_min',  'TMP_22.0_mean',  'O3_1.0_max',  'RH_48.0_mean',  'RH_max',  'RH_2.0_max',  'O3_2.0_max',  'O3_1.0_min',  'RH_23.0_min',  'RH_25.0_mean',  'TMP_mean_frcst_24',  'RH_0.0_mean',  'TMP_24.0_mean',  'hora_43.0',  'hora_20.0',  'O3_1.0_mean',  'TMP_mean',  'O3_44.0_mean',  'TMP_0.0_max',  'RH_0.0_max',  'O3_2.0_min',  'O3_0.0_max',  'O3_45.0_max',  'TMP_23.0_mean',  'RH_0.0_min',  'O3_23.0_max',  'RH_mean_frcst_22',  'TMP_24.0_max',  'RH_47.0_mean',  'TMP_mean_frcst_23',  'TMP_0.0_min',  'O3_24.0_mean',  'TMP_max',  'TMP_1.0_max',  'WSP_44.0_mean',  'O3_0.0_mean',  'O3_23.0_min',  'O3_46.0_max',  'RH_mean_frcst_21',  'RH_mean_frcst_1',  'RH_mean_frcst_2',  'TMP_21.0_max',  'TMP_mean_frcst_2',  'RH_2.0_min',  'TMP_46.0_mean',  'RH_24.0_min',  'TMP_45.0_mean',  'TMP_mean_frcst_3',  'TMP_mean_frcst_1',  'O3_min',  'O3_mean',  'O3_47.0_mean',  'TMP_2.0_mean',  'TMP_23.0_max',  'O3_2.0_mean',  'O3_21.0_max',  'RH_23.0_mean',  'RH_24.0_mean',  'O3_25.0_mean',  'O3_46.0_mean',  'RH_1.0_min',  'RH_45.0_mean',  'TMP_47.0_mean',  'RH_24.0_max',  'TMP_22.0_max',  'hora_19.0',  'O3_0.0_min',  'RH_mean_frcst_24',  'RH_mean',  'TMP_21.0_mean',  'RH_22.0_mean',  'RH_mean_frcst_3',  'RH_44.0_mean',  'TMP_min',  'TMP_0.0_mean',  'O3_24.0_max',  'hora_33.0',  'O3_21.0_mean',  'O3_22.0_min',  'hora_10.0',  'O3_22.0_mean',  'O3_24.0_min',  'RH_2.0_mean',  'RH_mean_frcst_23',  'O3_23.0_mean',  'O3_48.0_mean',  'RH_21.0_mean',  'RH_1.0_max',  'TMP_1.0_min',  'RH_1.0_mean',  'TMP_1.0_mean',  'RH_23.0_max',  'RH_46.0_mean',  'TMP_mean_frcst_22',  'hora_34.0',  'hora_9.0',  'O3_max',  'O3_45.0_mean',  'O3_22.0_max',  'RH_21.0_min',  'RH_min',  'WSP_mean']
    PM10mean_columns_final = ['PM10mean_0.0_mean',  'PM10mean_20.0_mean',  'PM10_8.0_mean',  'PM10mean_3.0_max',  'PM10mean_25.0_max',  'O3_max',  'PM10mean_18.0_mean',  'PM25mean_2.0_max',  'PM10mean_9.0_max',  'PM10_1.0_mean',  'PM10mean_18.0_max',  'PM10mean_10.0_max',  'PM25mean_5.0_max',  'PM10mean_33.0_max',  'PM10mean_15.0_max',  'PM10mean_6.0_max',  'PM10mean_24.0_mean',  'PM10mean_max',  'PM10_1.0_max',  'PM10mean_1.0_max',  'PM10_10.0_mean',  'PM10mean_7.0_mean',  'PM10mean_24.0_max',  'PM10mean_21.0_mean',  'PM10mean_37.0_max',  'PM10mean_23.0_mean',  'PM25mean_0.0_mean',  'PM10_4.0_max',  'PM10mean_17.0_max',  'PM25mean_6.0_max',  'PM10mean_32.0_max',  'PM10_5.0_max',  'PM25mean_1.0_max',  'PM10mean_6.0_mean',  'PM10mean_11.0_mean',  'PM10mean_26.0_max',  'PM10mean_22.0_mean',  'PM10mean_16.0_mean',  'PM10mean_8.0_mean',  'PM25mean_4.0_max',  'PM10mean_30.0_max',  'PM25mean_0.0_max',  'PM10mean_23.0_max',  'NO2_max',  'WSP_mean',  'PM10mean_9.0_mean',  'PM10mean_17.0_mean',  'PM25mean_3.0_mean',  'PM25mean_7.0_max',  'PM10mean_16.0_max',  'PM10mean_14.0_mean',  'PM10mean_2.0_max',  'PM10mean_2.0_mean',  'PM10mean_mean',  'PM10_0.0_max',  'PM10mean_12.0_max',  'PM10mean_13.0_mean',  'PM10mean_8.0_max',  'PM10mean_25.0_mean',  'PM10mean_27.0_max',  'PM10mean_31.0_max',  'PM10mean_0.0_max',  'PM25mean_3.0_max',  'PM10mean_14.0_max',  'PM25mean_1.0_mean',  'PM10_2.0_max',  'PM10_0.0_mean',  'PM10mean_15.0_mean',  'PM10mean_19.0_mean',  'PM10mean_13.0_max',  'PM10mean_4.0_max',  'PM10_3.0_max',  'PM10mean_1.0_mean',  'PM10_7.0_max',  'PM10mean_20.0_max',  'PM25mean_8.0_max',  'PM10_5.0_mean',  'PM10mean_21.0_max',  'PM10mean_29.0_max',  'PM10mean_12.0_mean',  'PM10_4.0_mean',  'PM10_mean',  'RH_mean',  'PM10mean_5.0_max',  'PM2.5_max',  'PM10mean_4.0_mean',  'PM10_6.0_max',  'PM10mean_36.0_max',  'PM10mean_7.0_max',  'PM10_7.0_mean',  'PM25mean_mean',  'PM10_9.0_mean',  'PM25mean_9.0_max',  'PM10_3.0_mean',  'PM2.5_0.0_max',  'PM10mean_3.0_mean',  'PM10_6.0_mean',  'PM10mean_5.0_mean',  'PM10mean_22.0_max',  'PM10_max',  'PM25mean_10.0_max',  'PM10mean_35.0_max',  'PM25mean_max',  'PM10mean_19.0_max',  'PM10mean_10.0_mean',  'PM10mean_11.0_max',  'PM25mean_2.0_mean',  'TMP_mean',  'PM10mean_34.0_max',  'PM10_2.0_mean',  'PM10mean_28.0_max']
    PM25mean_columns_final = ['PM25mean_14.0_max',  'PM10mean_0.0_mean',  'PM10mean_20.0_mean',  'PM10_8.0_mean',  'PM10mean_3.0_max',  'PM2.5_6.0_max',  'O3_max',  'PM10mean_18.0_mean',  'PM25mean_2.0_max',  'PM2.5_9.0_mean',  'PM10_1.0_mean',  'PM2.5_2.0_max',  'PM25mean_0.0_min',  'PM25mean_13.0_max',  'PM10mean_10.0_max',  'PM25mean_5.0_max',  'PM10mean_15.0_max',  'PM10mean_6.0_max',  'PM10mean_0.0_min',  'NO2_3.0_max',  'PM10mean_max',  'PM10_1.0_max',  'PM10mean_1.0_max',  'PM10_10.0_mean',  'PM2.5_4.0_mean',  'PM10mean_7.0_mean',  'PM10mean_21.0_mean',  'PM2.5_mean',  'PM10mean_23.0_mean',  'PM25mean_0.0_mean',  'PM10_4.0_max',  'PM10mean_17.0_max',  'PM10_12.0_mean',  'PM25mean_6.0_max',  'PM25mean_6.0_mean',  'PM25mean_1.0_max',  'PM10mean_6.0_mean',  'PM10mean_11.0_mean',  'PM2.5_3.0_max',  'PM2.5_2.0_mean',  'PM10mean_22.0_mean',  'PM10mean_16.0_mean',  'PM10mean_8.0_mean',  'PM25mean_11.0_mean',  'PM25mean_4.0_max',  'PM25mean_0.0_max',  'NO2_max',  'WSP_mean',  'PM10mean_9.0_mean',  'PM10mean_17.0_mean',  'PM2.5_8.0_mean',  'PM25mean_3.0_mean',  'PM25mean_7.0_max',  'PM25mean_9.0_mean',  'PM10mean_16.0_max',  'PM10mean_14.0_mean',  'NO2_mean',  'PM10mean_2.0_max',  'PM10mean_2.0_mean',  'PM10mean_mean',  'PM10_0.0_max',  'PM10mean_12.0_max',  'PM25mean_min',  'PM10mean_13.0_mean',  'PM10mean_8.0_max',  'PM10mean_0.0_max',  'PM25mean_3.0_max',  'PM10mean_14.0_max',  'PM25mean_1.0_mean',  'PM10_2.0_max',  'PM10_0.0_mean',  'PM10mean_15.0_mean',  'PM10mean_19.0_mean',  'PM10_11.0_mean',  'PM10mean_13.0_max',  'PM10mean_4.0_max',  'PM25mean_10.0_mean',  'PM25mean_14.0_mean',  'PM10_3.0_max',  'PM10mean_1.0_mean',  'PM2.5_1.0_mean',  'PM25mean_8.0_max',  'PM25mean_4.0_mean',  'PM10_5.0_mean',  'PM25mean_5.0_mean',  'PM25mean_11.0_max',  'PM10mean_12.0_mean',  'PM10_4.0_mean',  'PM10_mean',  'RH_mean',  'PM10mean_1.0_min',  'PM2.5_0.0_mean',  'PM10mean_5.0_max',  'PM2.5_max',  'PM10mean_4.0_mean',  'PM2.5_4.0_max',  'PM2.5_5.0_max',  'PM2.5_min',  'PM10mean_7.0_max',  'PM10_7.0_mean',  'PM25mean_mean',  'PM10_9.0_mean',  'PM25mean_9.0_max',  'PM10mean_min',  'PM10_3.0_mean',  'PM2.5_1.0_max',  'PM2.5_5.0_mean',  'PM2.5_6.0_mean',  'PM2.5_0.0_max',  'PM10mean_3.0_mean',  'PM10_6.0_mean',  'PM10mean_5.0_mean',  'PM25mean_8.0_mean',  'PM10_max',  'PM25mean_10.0_max',  'PM25mean_max',  'PM25mean_12.0_max',  'PM25mean_1.0_min',  'PM2.5_3.0_mean',  'PM25mean_15.0_mean',  'PM10mean_10.0_mean',  'PM25mean_12.0_mean',  'PM10mean_11.0_max',  'PM25mean_2.0_mean',  'TMP_mean',  'PM2.5_7.0_mean',  'PM25mean_7.0_mean',  'PM25mean_13.0_mean',  'PM10_2.0_mean',  'PM10mean_9.0_max',  'PM2.5_0.0_min']

    #Columnas con losque se filtra inicialmente:

    PM25mean_columns_filter = ['PM25mean_0.0',  'PM10_0.0',  'PM2.5_0.0',  'PM10mean_0.0',  'PM25mean_0.0',  'PM10mean_1.0',  'PM10mean_2.0',  'PM10mean_3.0',  'PM10mean_4.0',  'PM10mean_5.0',  'PM10mean_6.0',  'PM10mean_7.0',  'PM10mean_8.0',  'PM10mean_9.0',  'PM10mean_10.0',  'PM10mean_11.0',  'PM10mean_12.0',  'PM10mean_13.0',  'PM10mean_14.0',  'PM10mean_15.0',  'PM10mean_16.0',  'PM25mean_1.0',  'PM25mean_2.0',  'PM25mean_3.0',  'PM25mean_4.0',  'PM25mean_5.0',  'PM25mean_6.0',  'PM25mean_7.0',  'PM25mean_8.0',  'PM25mean_9.0',  'PM25mean_10.0',  'PM25mean_11.0',  'PM25mean_12.0',  'PM25mean_13.0',  'PM25mean_14.0',  'PM25mean_15.0',  'PM25mean_16.0',  'PM25mean_17.0',  'PM25mean_18.0',  'PM10mean_1.0',  'PM10mean_2.0',  'PM10mean_3.0',  'PM10mean_4.0',  'PM10mean_5.0',  'PM10mean_6.0',  'PM10mean_7.0',  'PM10mean_8.0',  'PM10mean_9.0',  'PM10mean_10.0',  'PM10mean_11.0',  'PM10mean_12.0',  'PM10mean_13.0',  'PM10mean_14.0',  'PM10mean_15.0',  'PM10mean_16.0',  'PM10mean_17.0',  'PM10mean_18.0',  'PM10mean_19.0',  'PM10mean_20.0',  'PM10mean_21.0',  'PM10mean_22.0',  'PM10mean_23.0',  'PM10mean_24.0',  'PM10mean_25.0',  'PM10mean_26.0',  'PM10mean_27.0',  'PM10mean_28.0',  'PM10mean_29.0',  'PM10mean_30.0',  'PM10mean_31.0',  'PM10mean_32.0',  'PM10mean_33.0',  'PM10mean_34.0',  'PM10mean_35.0',  'PM10mean_36.0',  'PM10mean_37.0',  'PM10mean_38.0',  'PM25mean_1.0',  'PM25mean_2.0',  'PM25mean_3.0',  'PM25mean_4.0',  'PM25mean_5.0',  'PM25mean_6.0',  'PM25mean_7.0',  'PM25mean_8.0',  'PM25mean_9.0',  'PM25mean_10.0',  'PM25mean_11.0',  'PM25mean_12.0',  'PM25mean_13.0',  'PM25mean_14.0',  'PM25mean_15.0',  'PM25mean_16.0',  'PM25mean_17.0',  'PM25mean_18.0',  'PM25mean_19.0',  'PM25mean_20.0',  'PM25mean_21.0',  'PM25mean_22.0',  'PM25mean_23.0',  'PM25mean_24.0',  'PM25mean_25.0',  'PM25mean_26.0',  'PM25mean_27.0',  'PM25mean_28.0',  'PM25mean_29.0',  'PM25mean_30.0',  'PM25mean_31.0',  'PM25mean_32.0',  'PM25mean_33.0',  'PM25mean_34.0',  'PM25mean_35.0',  'PM25mean_36.0',  'PM25mean_37.0',  'NO2_3.0',  'NO2_4.0',  'NO2_5.0',  'NO2_6.0',  'NO2_7.0',  'NO2_8.0',  'NO2_9.0',  'NO2_10.0',  'NO2_11.0',  'NO2_12.0',  'NO2_13.0',  'NO2_14.0',  'NO2_15.0',  'NO2_16.0',  'NO2_17.0',  'NO2_18.0',  'NO2_19.0',  'NO2_20.0',  'NO2_21.0',  'NO2_22.0',  'NO2_23.0',  'PM10_1.0',  'PM10_2.0',  'PM10_3.0',  'PM10_4.0',  'PM10_5.0',  'PM10_6.0',  'PM10_7.0',  'PM10_8.0',  'PM10_9.0',  'PM10_10.0',  'PM10_11.0',  'PM10_12.0',  'PM10_13.0',  'PM10_14.0',  'PM10_15.0',  'PM10_16.0',  'PM10_17.0',  'PM10_18.0',  'PM10_19.0',  'PM10_20.0',  'PM10_21.0',  'PM10_22.0',  'PM10_23.0',  'PM10_24.0',  'PM10_25.0',  'PM10_26.0',  'PM10_27.0',  'PM10_28.0',  'PM10_29.0',  'PM10_30.0',  'PM2.5_1.0',  'PM2.5_2.0',  'PM2.5_3.0',  'PM2.5_4.0',  'PM2.5_5.0',  'PM2.5_6.0',  'PM2.5_7.0',  'PM2.5_8.0',  'PM2.5_9.0',  'PM2.5_10.0',  'PM2.5_11.0',  'PM2.5_12.0',  'PM2.5_13.0',  'PM2.5_14.0',  'PM2.5_15.0',  'PM2.5_16.0',  'PM2.5_17.0',  'PM2.5_18.0',  'PM2.5_19.0',  'PM2.5_20.0',  'PM2.5_21.0',  'PM2.5_22.0',  'PM2.5_23.0',  'PM2.5_24.0',  'PM2.5_25.0',  'PM2.5_26.0',  'PM2.5_27.0',  'PM2.5_28.0',  'PM10mean_1.0',  'PM10mean_2.0',  'PM10mean_3.0',  'PM10mean_4.0',  'PM10mean_5.0',  'PM10mean_6.0',  'PM10mean_7.0',  'PM10mean_8.0',  'PM10mean_9.0',  'PM10mean_10.0',  'PM10mean_11.0',  'PM10mean_12.0',  'PM10mean_13.0',  'PM10mean_14.0',  'PM10mean_15.0',  'PM10mean_16.0',  'PM10mean_17.0',  'PM10mean_18.0',  'PM10mean_19.0',  'PM10mean_20.0',  'PM10mean_21.0',  'PM10mean_22.0',  'PM10mean_23.0',  'PM10mean_24.0',  'PM10mean_25.0',  'PM10mean_26.0',  'PM10mean_27.0',  'PM10mean_28.0',  'PM10mean_29.0',  'PM10mean_30.0',  'PM10mean_31.0',  'PM10mean_32.0',  'PM10mean_33.0',  'PM10mean_34.0',  'PM10mean_35.0',  'PM10mean_36.0',  'PM10mean_37.0',  'PM10mean_38.0',  'PM10mean_39.0',  'PM10mean_40.0',  'PM10mean_41.0',  'PM10mean_42.0',  'PM25mean_1.0',  'PM25mean_2.0',  'PM25mean_3.0',  'PM25mean_4.0',  'PM25mean_5.0',  'PM25mean_6.0',  'PM25mean_7.0',  'PM25mean_8.0',  'PM25mean_9.0',  'PM25mean_10.0',  'PM25mean_11.0',  'PM25mean_12.0',  'PM25mean_13.0',  'PM25mean_14.0',  'PM25mean_15.0',  'PM25mean_16.0',  'PM25mean_17.0',  'PM25mean_18.0',  'PM25mean_19.0',  'PM25mean_20.0',  'PM25mean_21.0',  'PM25mean_22.0',  'PM25mean_23.0',  'PM25mean_24.0',  'PM25mean_25.0',  'PM25mean_26.0',  'PM25mean_27.0',  'PM25mean_28.0',  'PM25mean_29.0',  'PM25mean_30.0',  'PM25mean_31.0',  'PM25mean_32.0',  'PM25mean_33.0',  'PM25mean_34.0']
    PM10mean_columns_filter = ['PM10_0.0',  'PM2.5_0.0',  'PM10mean_0.0',  'PM25mean_0.0',  'PM10mean_1.0',  'PM10mean_2.0',  'PM10mean_3.0',  'PM10mean_4.0',  'PM10mean_5.0',  'PM10mean_6.0',  'PM10mean_7.0',  'PM10mean_8.0',  'PM10mean_9.0',  'PM10mean_10.0',  'PM10mean_11.0',  'PM10mean_12.0',  'PM10mean_13.0',  'PM10mean_14.0',  'PM10mean_15.0',  'PM25mean_1.0',  'PM25mean_2.0',  'PM10mean_1.0',  'PM10mean_2.0',  'PM10mean_3.0',  'PM10mean_4.0',  'PM10mean_5.0',  'PM10mean_6.0',  'PM10mean_7.0',  'PM10mean_8.0',  'PM10mean_9.0',  'PM10mean_10.0',  'PM10mean_11.0',  'PM10mean_12.0',  'PM10mean_13.0',  'PM10mean_14.0',  'PM10mean_15.0',  'PM10mean_16.0',  'PM10mean_17.0',  'PM10mean_18.0',  'PM10mean_19.0',  'PM10mean_20.0',  'PM10mean_21.0',  'PM10mean_22.0',  'PM10mean_23.0',  'PM10mean_24.0',  'PM10mean_25.0',  'PM10mean_26.0',  'PM10mean_27.0',  'PM10mean_28.0',  'PM10mean_29.0',  'PM10mean_30.0',  'PM10mean_31.0',  'PM10mean_32.0',  'PM10mean_33.0',  'PM10mean_34.0',  'PM10mean_35.0',  'PM10mean_36.0',  'PM10mean_37.0',  'PM10mean_38.0',  'PM10mean_39.0',  'PM10mean_40.0',  'PM10mean_41.0',  'PM10mean_42.0',  'PM10mean_43.0',  'PM10mean_44.0',  'PM10mean_45.0',  'PM10mean_46.0',  'PM10mean_47.0',  'PM10mean_48.0',  'PM25mean_1.0',  'PM25mean_2.0',  'PM25mean_3.0',  'PM25mean_4.0',  'PM25mean_5.0',  'PM25mean_6.0',  'PM25mean_7.0',  'PM25mean_8.0',  'PM25mean_9.0',  'PM25mean_10.0',  'PM25mean_11.0',  'PM25mean_12.0',  'PM25mean_13.0',  'PM25mean_14.0',  'PM25mean_15.0',  'PM25mean_16.0',  'PM25mean_17.0',  'PM25mean_18.0',  'PM25mean_19.0',  'PM25mean_20.0',  'PM25mean_21.0',  'PM25mean_22.0',  'PM25mean_23.0',  'PM25mean_24.0',  'PM25mean_25.0',  'PM25mean_26.0',  'PM25mean_27.0',  'PM25mean_28.0',  'PM25mean_29.0',  'PM25mean_30.0',  'PM25mean_31.0',  'PM25mean_32.0',  'PM25mean_33.0',  'PM25mean_34.0',  'PM25mean_35.0',  'PM25mean_36.0',  'PM25mean_37.0',  'NO2_5.0',  'NO2_6.0',  'NO2_7.0',  'NO2_8.0',  'NO2_9.0',  'NO2_10.0',  'NO2_11.0',  'NO2_12.0',  'NO2_13.0',  'NO2_14.0',  'NO2_15.0',  'NO2_16.0',  'NO2_17.0',  'NO2_18.0',  'NO2_19.0',  'NO2_20.0',  'NO2_21.0',  'NO2_22.0',  'NO2_23.0',  'PM10_1.0',  'PM10_2.0',  'PM10_3.0',  'PM10_4.0',  'PM10_5.0',  'PM10_6.0',  'PM10_7.0',  'PM10_8.0',  'PM10_9.0',  'PM10_10.0',  'PM10_11.0',  'PM10_12.0',  'PM10_13.0',  'PM10_14.0',  'PM10_15.0',  'PM10_16.0',  'PM10_17.0',  'PM10_18.0',  'PM10_19.0',  'PM10_20.0',  'PM10_21.0',  'PM10_22.0',  'PM10_23.0',  'PM10_24.0',  'PM10_25.0',  'PM10_26.0',  'PM10_27.0',  'PM10_28.0',  'PM10_29.0',  'PM10_30.0',  'PM10_31.0',  'PM10_32.0',  'PM10_33.0',  'PM10_34.0',  'PM10_35.0',  'PM10_36.0',  'PM2.5_1.0',  'PM2.5_2.0',  'PM2.5_3.0',  'PM2.5_4.0',  'PM2.5_5.0',  'PM2.5_6.0',  'PM2.5_7.0',  'PM2.5_8.0',  'PM2.5_9.0',  'PM2.5_10.0',  'PM2.5_11.0',  'PM2.5_12.0',  'PM2.5_13.0',  'PM2.5_14.0',  'PM2.5_15.0',  'PM2.5_16.0',  'PM2.5_17.0',  'PM2.5_18.0',  'PM2.5_19.0',  'PM2.5_20.0',  'PM2.5_21.0',  'PM2.5_22.0',  'PM2.5_23.0',  'PM10mean_1.0',  'PM10mean_2.0',  'PM10mean_3.0',  'PM10mean_4.0',  'PM10mean_5.0',  'PM10mean_6.0',  'PM10mean_7.0',  'PM10mean_8.0',  'PM10mean_9.0',  'PM10mean_10.0',  'PM10mean_11.0',  'PM10mean_12.0',  'PM10mean_13.0',  'PM10mean_14.0',  'PM10mean_15.0',  'PM10mean_16.0',  'PM10mean_17.0',  'PM10mean_18.0',  'PM10mean_19.0',  'PM10mean_20.0',  'PM10mean_21.0',  'PM10mean_22.0',  'PM10mean_23.0',  'PM10mean_24.0',  'PM10mean_25.0',  'PM10mean_26.0',  'PM10mean_27.0',  'PM10mean_28.0',  'PM10mean_29.0',  'PM10mean_30.0',  'PM10mean_31.0',  'PM10mean_32.0',  'PM10mean_33.0',  'PM10mean_34.0',  'PM10mean_35.0',  'PM10mean_36.0',  'PM10mean_37.0',  'PM10mean_38.0',  'PM10mean_39.0',  'PM10mean_40.0',  'PM10mean_41.0',  'PM10mean_42.0',  'PM10mean_43.0',  'PM10mean_44.0',  'PM10mean_45.0',  'PM10mean_46.0',  'PM10mean_47.0',  'PM10mean_48.0',  'PM25mean_1.0',  'PM25mean_2.0',  'PM25mean_3.0',  'PM25mean_4.0',  'PM25mean_5.0',  'PM25mean_6.0',  'PM25mean_7.0',  'PM25mean_8.0',  'PM25mean_9.0',  'PM25mean_10.0',  'PM25mean_11.0',  'PM25mean_12.0',  'PM25mean_13.0',  'PM25mean_14.0',  'PM25mean_15.0',  'PM25mean_16.0',  'PM25mean_17.0',  'PM25mean_18.0',  'PM25mean_19.0',  'PM25mean_20.0',  'PM25mean_21.0',  'PM25mean_22.0',  'PM25mean_23.0',  'PM25mean_24.0',  'PM25mean_25.0',  'PM25mean_26.0',  'PM25mean_27.0',  'PM25mean_28.0',  'PM25mean_29.0']
    O3_columns_filter = ['O3_0.0',  'RH_0.0',  'TMP_0.0',  'hora_9.0',  'hora_10.0',  'hora_11.0',  'hora_12.0',  'hora_13.0',  'hora_18.0',  'hora_19.0',  'hora_20.0',  'hora_21.0',  'hora_22.0',  'hora_32.0',  'hora_33.0',  'hora_34.0',  'hora_35.0',  'hora_36.0',  'hora_42.0',  'hora_43.0',  'hora_44.0',  'hora_45.0',  'O3_1.0',  'O3_2.0',  'O3_3.0',  'O3_4.0',  'O3_21.0',  'O3_22.0',  'O3_23.0',  'O3_24.0',  'O3_25.0',  'O3_26.0',  'O3_27.0',  'O3_45.0',  'O3_46.0',  'O3_47.0',  'O3_48.0',  'O3_1.0',  'O3_2.0',  'O3_3.0',  'O3_4.0',  'O3_20.0',  'O3_21.0',  'O3_22.0',  'O3_23.0',  'O3_24.0',  'O3_25.0',  'O3_26.0',  'O3_27.0',  'O3_44.0',  'O3_45.0',  'O3_46.0',  'O3_47.0',  'O3_48.0',  'CO_5.0',  'CO_6.0',  'CO_7.0',  'CO_8.0',  'CO_29.0',  'CO_30.0',  'CO_31.0',  'NO_6.0',  'NO_7.0',  'NO_8.0',  'NO_9.0',  'NO_30.0',  'NO_31.0',  'NO_32.0',  'NOX_6.0',  'NOX_7.0',  'NOX_8.0',  'NOX_9.0',  'NOX_30.0',  'NOX_31.0',  'NOX_32.0',  'O3_1.0',  'O3_2.0',  'O3_3.0',  'O3_4.0',  'O3_21.0',  'O3_22.0',  'O3_23.0',  'O3_24.0',  'O3_25.0',  'O3_26.0',  'O3_27.0',  'O3_44.0',  'O3_45.0',  'O3_46.0',  'O3_47.0',  'O3_48.0',  'PM10_1.0',  'PM10_2.0',  'PM10_3.0',  'PM10_4.0',  'PM10_5.0',  'PM10_6.0',  'PM10_26.0',  'PM10_27.0',  'PM10_28.0',  'PM10_29.0',  'PM2.5_1.0',  'PM2.5_2.0',  'PM2.5_3.0',  'PM2.5_4.0',  'PM2.5_5.0',  'PM2.5_6.0',  'PM2.5_26.0',  'PM2.5_27.0',  'PM2.5_28.0',  'PM2.5_29.0',  'RH_1.0',  'RH_2.0',  'RH_3.0',  'RH_20.0',  'RH_21.0',  'RH_22.0',  'RH_23.0',  'RH_24.0',  'RH_25.0',  'RH_26.0',  'RH_44.0',  'RH_45.0',  'RH_46.0',  'RH_47.0',  'RH_48.0',  'TMP_1.0',  'TMP_2.0',  'TMP_3.0',  'TMP_20.0',  'TMP_21.0',  'TMP_22.0',  'TMP_23.0',  'TMP_24.0',  'TMP_25.0',  'TMP_26.0',  'TMP_44.0',  'TMP_45.0',  'TMP_46.0',  'TMP_47.0',  'TMP_48.0',  'WSP_8.0',  'WSP_9.0',  'WSP_10.0',  'WSP_11.0',  'WSP_44.0',  'WSP_45.0']

    if contaminante == "O3":
        columns = O3_columns_final
        columns_filter = O3_columns_filter
    elif contaminante=="PM10mean":
        columns = PM10mean_columns_final
        columns_filter = PM10mean_columns_filter
    elif contaminante=="PM25mean":
        columns = PM25mean_columns_final
        columns_filter = PM25mean_columns_filter

    #columns_filter

    ## 1. 
    ### Usando el archivo de correlación con retraso temporal, se crean nuevas columnas con el atributo y
    #las horas atrás.


    #La corraloción con retraso temporal se refiere a la influencia que tienen las condiciones atmosféricas
    #de un hora dada sobre las condiciones futuras, por ejemplo, si el nivel de contaminación de actual es 
    #alto ¿este valor influirá en la contaminación del día siguiente o la contaminación de cada día no se
    #ve influida por valores pasados? La respuesta es la primera opción, valores pasados de las condiciones
    #atmosféricas influyen fuertemente en la contaminación _actual_.

    #Esta propiedad es clave para la generación de un modelo estadístico, la existencia de patrones y de
    #factores que aparentemente influyen en la contaminación nos permiten tener un conjunto de datos que 
    #determinen la contaminación.


    #Ya que tenemos los modelos entrenados, podemos definir el nombre de las columnas para posteriormente 
    #realizar el desplazamiento temporal:

    O3_wo_end = columns_filter

    nombres_items = [item.split("_")[0] for item in O3_wo_end]
    horas_items = [item.split("_")[1] for item in O3_wo_end]

    #Una vez que tenemos los contaminantes y el número de horas de retraso, definimos una función que 
    #nos genere una nueva columna con el contaminante recorrido temporalmente las horas indicadas 
    #en **O3_corr_filtro**:

    def shit_corr(df):
        for i in range(len(O3_wo_end)):
            name_column = str(nombres_items[i] + "_" + str(horas_items[i]))
                                                    # se define el nombre de la columna a crear con el formato:
                                                # contaminante _ horas de retraso
            df[name_column] = df[nombres_items[i]].shift(
                int(float(str(horas_items[i])))) # se toma el nombre del contaminante y se 
                                                                # desplaza temporalmente las horas indicadas
        return df.dropna()

    #Debido a que por cada hora se tienen tantos registros como el número de estaciones de monitoreo,
    #existen fechas duplicadas en el df:

    #Al tener fechas duplicadas, puede ocurrrir errores al aplicar el desplazamiento, por este motivo
    #vamos a generar **un dataframe por cada estación**, aplicaremos la función y por último uniremos 
    #los dataframes:

    estaciones = data_hour_merge_24.id_station.unique().tolist() # recolectamos los nombres de las estaciones
    data_est = {}
    for elem in estaciones:
        data_est[elem] = data_hour_merge_24[data_hour_merge_24.id_station == elem] #hacemos un df por estación

    #Aplicamos la función para cada dataframe:

    data_shift = {}
    for elem in data_est:
        data_shift[elem] = shit_corr(data_est[elem])

    #Unimos los dataframes:

    df_append = pd.DataFrame(columns = data_est["MER"].columns.tolist())
    for key in data_est:
        df_append = df_append.append(data_est[key], ignore_index=True,sort=True)


    # 2.
    ### Se obtiene el promedio, máximo y mínimo por hora de todas las estaciones.

    #Debido a que solo algunas estaciones miden todos los atributos con los que se está trabajando, 
    #se encontró que agrupar las condiciones atmosféricas de cada atributo con el promedio, máximo y 
    #mínimo, genera una mejora considerable al desempeño final de los modelos, debido a que al hacer
    #esta agrupación se suavizan los valores y también se homogeniza los atributos de cada estación.

    #Se guardan las columnas al aplicar el groupby y se elimina "fecha":

    cols = df_append.columns.tolist() 
    cols.remove("fecha")
    cols.remove("id_station")
    df_append[cols] = df_append[cols].astype(float)

    #Aplicamos los groupbys:

    data_hour_merge_24_max = df_append.groupby('fecha')[cols].max()
    data_hour_merge_24_max.reset_index(inplace=True)

    data_hour_merge_24_min = df_append.groupby('fecha')[cols].min()
    data_hour_merge_24_min.reset_index(inplace=True)

    data_hour_merge_24_mean = df_append.groupby('fecha')[cols].mean()
    data_hour_merge_24_mean.reset_index(inplace=True)

    #### Cambiamos los nombres de las estaciones:
    #Debido a que cada groupby genera los mismos nombres, esto puede generar problemas para 
    #identificar de qué columna se trata, se cambiarán los nombres a "contaminante_max", 
    #"contaminante_min", "contaminante_mean" según corresponda.

    #Se obtienenn los nonbres de las columnas en forma de lista:

    max_columns = data_hour_merge_24_max.columns.tolist()
    min_columns = data_hour_merge_24_min.columns.tolist()
    mean_columns = data_hour_merge_24_mean.columns.tolist()

    #Se cambian los nombres en las listas:

    for i in range(len(max_columns)):
        if not max_columns[i].startswith("fecha") \
        and not max_columns[i].startswith("hora") \
        and not max_columns[i].startswith("dia") \
        and not max_columns[i].startswith("mes"):
            max_columns[i] = max_columns[i]+"_max"

        if not min_columns[i].startswith("fecha") \
        and not min_columns[i].startswith("hora") \
        and not min_columns[i].startswith("dia") \
        and not min_columns[i].startswith("mes"):
            min_columns[i] = min_columns[i]+"_min" 

        if not mean_columns[i].startswith("fecha") \
        and not mean_columns[i].startswith("hora") \
        and not mean_columns[i].startswith("dia") \
        and not mean_columns[i].startswith("mes"):
            mean_columns[i] = mean_columns[i]+"_mean"

    #Se renombra las columnas de los dataframes

    data_hour_merge_24_mean.columns = mean_columns
    data_hour_merge_24_min.columns = min_columns
    data_hour_merge_24_max.columns = max_columns

    #Unimos los df's 

    data_hour_merge = pd.merge(data_hour_merge_24_mean, data_hour_merge_24_max, on=['fecha'])
    data_hour_merge = pd.merge(data_hour_merge, data_hour_merge_24_min, on=['fecha'])

    data_hour_merge_columns = data_hour_merge.columns.tolist()

    #Debido a que el proceso de obtener el máximo y mínimo es lento, es conveniente guardar el
    #archivo de manera provisional:

    # Arreglemos los nnombres:

    for item in range(len(data_hour_merge_columns)):
        if  data_hour_merge_columns[item].endswith("_x")\
        or  data_hour_merge_columns[item].endswith("_y"):
            data_hour_merge_columns[item] = data_hour_merge_columns[item][:-2]

    data_hour_merge.columns = data_hour_merge_columns#.tolist()

    data_hour_merge = data_hour_merge.loc[:,~data_hour_merge.columns.duplicated()]

    data_hour_merge = data_hour_merge.sort_values(['fecha',"hora"], ascending=[0,1]).reset_index(drop=True)

    ## 3. 
    ### Generar las columna a pronosticar y obtener la correlación de los valores promedio máximo y 
    #mínimo con éste. Filtrar los valores más importantes.


    #Ahora vamos a sacar los atributos más correlacionado con el contaminante a pronosticar a pronosticar. 

    #data_hour_merge.head()

    ## 4. 
    ### Generar las columnas de pronóstico de RH, WSP TMP.

    #Se observó que usando datos de pronóstico atmosférico, el modelo tiene un mejor desempeño,
    #vamos a añadir "RH_mean","WSP_mean","TMP_mean" a los atributos con un pronóstico de 24 horas.

    lista_frcst = ["RH_mean","WSP_mean","TMP_mean"]

    for item_pronostico in lista_frcst:
        for i in range(1, 25):
            col_name = str(item_pronostico + "_frcst_" + str(i))
            data_hour_merge[col_name] = data_hour_merge[item_pronostico].shift(i)
    data_hour_merge.dropna(inplace=True)

    
        #### Columnas de pronóstico ## solo para datos de verificación
    
    if contaminante == 'PM10' or contaminante == 'PM2.5':
        item = contaminante+"mean_max"
    else:
        item = contaminante+"_max" 
    print(item)
    
    for i in range(1, 25):
        col_name = str(item+"_frcst_"+str(i))
        columns.append(col_name)
        data_hour_merge[col_name] = data_hour_merge[item].shift(i)
    data_hour_merge.dropna(inplace=True)
    
    
            ### columnas de pronóstico
    
    
    columns.append("fecha")
    data_hour_merge = data_hour_merge[columns]

    #path_final = str("./datos/ingesta_modelos_"+ contaminante + ".csv")
    path_final = str("./verificacion/verifcacion_modelos_"+ contaminante + ".csv")#verificacion
    


    data_hour_merge.to_csv(path_final,sep=',', encoding='utf-8',index=False)

Test